In [ ]:
import os
import json
import pandas as pd
from openai import OpenAI
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
from google.colab import userdata

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
messages = pd.read_csv("messages.csv")
messages['id'] = messages.index
messages = messages[['id', 'Сообщения']]
messages.head()

,id,Сообщения
0,0,Пахота зяби под мн тр\nПо Пу 26/488\nОтд 12 26...
1,1,Пахота зяби под мн тр\nПо Пу 26/514\nОтд 12 26...
2,2,Пахота зяби под сою\nПо Пу 29/1367\nОтд 16 29/...
3,3,12.10\nВнесение мин удобрений под оз пшеницу 2...
4,4,Пахота зяби под сою\nПо Пу 15/1382\nОтд 16 15/...


In [ ]:
with open('abbreviations.json', 'r', encoding='utf-8') as file:
    abbreviations = json.load(file)

In [ ]:
def get_structured_data(message, msg_id, abbreviations):
    """
    Извлекает структурированные данные из сообщения с помощью LLM, используя Structured Output
    """
    abbreviations_str = json.dumps(abbreviations, ensure_ascii=False)

    system_prompt = """
    Ты — эксперт по обработке сельскохозяйственных данных. Твоя задача — извлечь структурированные данные из текстовых сообщений о полевых работах, строго следуя предоставленным инструкциям и словарю сокращений. Возвращай данные только в указанном формате, игнорируя нерелевантную информацию. Если данные отсутствуют или не могут быть точно извлечены, возвращай пустой список JSON объектов.
    """

    user_prompt = f"""
    ### Задача
    Извлеки из сообщения информацию о полевых работах и представь её в виде списка JSON-объектов. Каждый объект представляет одну уникальную операцию на уровне Производственного Участка (ПУ).

    ### Словарь сокращений
    Используй следующий словарь для расшифровки аббревиатур операций и культур, а также для определения принадлежности ПУ/Отделений к Подразделениям:
    {abbreviations_str}

    ### Правила извлечения данных для каждого объекта:

    1.  **Идентификация Объекта:**
        * Каждый JSON-объект должен соответствовать **одной операции, выполняемой на уровне Производственного Участка (ПУ)**.
        * Информация по Отделениям (Отд) сама по себе **не создает** новый объект, если для той же операции есть данные по ПУ.

    2.  **Заполнение полей:** Для каждой идентифицированной операции (на уровне ПУ) заполни следующие поля:
        * **Дата**: дата в формате "ДД/ММ" или "ДД/ММ/ГГГГ", если указана, иначе "".
        * **Подразделение**:
            * Определи подразделение ("АОР", "Восход", "ТСК", "Мир" и т.д.) на основе номера ПУ или Отд из сообщения, используя словарь.
            * **Приоритет:** Всегда используй данные по ПУ для определения операции.
            * **Исключение:** Если для какой-то операции данные по ПУ **полностью отсутствуют** (нет строки "По ПУ"), **только тогда** создай объект на основе данных Отделения (Отд), и в этом случае в поле "Подразделение" укажи "Отд [номер]" (например, "Отд 17").
        * **Операция**:
            * Полное название операции, расшифрованное с помощью словаря (например, "Пахота", "Дискование 2-е").
        * **Культура**:
            * Полное название культуры, расшифрованное с помощью словаря.
            * **ВАЖНОЕ ПРАВИЛО "ПОД":** Если в описании операции указано "[культура1] под [культура2]" (например, "сах св под пш"), то **ОСНОВНОЙ КУЛЬТУРОЙ СЧИТАЕТСЯ [культура2]** (в примере - "пш", т.е. "Пшеница озимая товарная"). Игнорируй [культуру1] ("сах св") при определении культуры для этого объекта.
            * Если указано просто "[операция] [культура]" (например, "Пахота соя"), используй эту культуру ("Соя товарная").
        * **За день (га)**:
            * Площадь за день в гектарах.
            * **ПРИОРИТЕТ ПУ:** **Всегда** извлекай это значение из строки `По ПУ [за день]/[с начала] га` или `По ПУ [за день] га/ с нарастающим [с начала] га`.
            * **Исключение:** Если строка "По ПУ" для данной операции отсутствует, **только тогда** используй значение из строки `Отд [номер] [за день]/[с начала] га`.
        * **С начала операции (га)**:
            * Общая площадь с начала операции в гектарах.
            * **ПРИОРИТЕТ ПУ:** **Всегда** извлекай это значение из строки `По ПУ [за день]/[с начала] га` или `По ПУ [за день] га/ с нарастающим [с начала] га`.
            * **Исключение:** Если строка "По ПУ" для данной операции отсутствует, **только тогда** используй значение из строки `Отд [номер] [за день]/[с начала] га`.
        * **Вал за день (ц)**:
            * Валовой сбор за день в центнерах (ц). Извлекай из "Вал [за день]/[с начала] кг".
            * **Перевод единиц:** Указанные в сообщении значения "Вал" даны в килограммах (кг). **ОБЯЗАТЕЛЬНО** переведи их в центнеры (ц), разделив на 100 (1 ц = 100 кг). Например, "Вал 1500/..." кг -> "Вал за день (ц)": "15".
            * Если данные отсутствуют, ставь "".
        * **Вал с начала (ц)**:
            * Общий валовой сбор с начала операции в центнерах (ц). Извлекай из "Вал [за день]/[с начала] кг".
            * **Перевод единиц:** Указанные в сообщении значения "Вал" даны в килограммах (кг). **ОБЯЗАТЕЛЬНО** переведи их в центнеры (ц), разделив на 100 (1 ц = 100 кг). Например, "Вал .../3000" кг -> "Вал с начала (ц)": "30".
            * Если данные отсутствуют, ставь "".

    ### Пример сообщения и ожидаемого вывода:

    **Сообщение:**
    ```
    Пахота зяби под сою
    По ПУ 7/1402
    Отд 17 7/141

    Вырав-ие зяби под кук/силос
    По ПУ 16/16
    Отд 12 16/16

    2-е диск сах св под пш
    По ПУ 59/1041
    Отд 17 59/349

    Только Отделение Опер
    Отд 5 10/100
    ```

    *(Предположим, словарь содержит: "Пахота зяби": "Пахота", "соя": "Соя товарная", "Вырав-ие зяби": "Выравнивание зяби", "кук/силос": "Кукуруза кормовая", "2-е диск": "Дискование 2-е", "сах св": "Свекла сахарная", "пш": "Пшеница озимая товарная", "Только Отделение Опер": "Тестовая операция", Отд 17, Отд 12 -> "АОР", Отд 5 -> "Восход")*

    **Ожидаемый вывод:**
    ```json
    [
        {{
            "Дата": "",
            "Подразделение": "АОР", // Взято из словаря по ПУ/Отд 17
            "Операция": "Пахота",
            "Культура": "Соя товарная", // "под сою"
            "За день (га)": "7", // Из ПУ
            "С начала операции (га)": "1402", // Из ПУ
            "Вал за день (ц)": "",
            "Вал с начала (ц)": ""
        }},
        {{
            "Дата": "",
            "Подразделение": "АОР", // Взято из словаря по ПУ/Отд 12
            "Операция": "Выравнивание зяби",
            "Культура": "Кукуруза кормовая", // "под кук/силос"
            "За день (га)": "16", // Из ПУ
            "С начала операции (га)": "16", // Из ПУ
            "Вал за день (ц)": "",
            "Вал с начала (ц)": ""
        }},
        {{
            "Дата": "",
            "Подразделение": "АОР", // Взято из словаря по ПУ/Отд 17
            "Операция": "Дискование 2-е",
            "Культура": "Пшеница озимая товарная", // ВАЖНО: из "под пш", а не "сах св"
            "За день (га)": "59", // Из ПУ
            "С начала операции (га)": "1041", // Из ПУ
            "Вал за день (ц)": "",
            "Вал с начала (ц)": ""
        }},
        {{
            "Дата": "",
            "Подразделение": "Отд 5", // Исключение: Нет данных по ПУ, используем Отд
            "Операция": "Тестовая операция",
            "Культура": "", // Не указана
            "За день (га)": "10", // Из Отд 5 (т.к. нет ПУ)
            "С начала операции (га)": "100", // Из Отд 5 (т.к. нет ПУ)
            "Вал за день (ц)": "",
            "Вал с начала (ц)": ""
        }}
    ]
    ```

    ### Обработай следующее сообщение:
    {message}
    """

    response_schema = {
        "type": "object",
        "properties": {
            "operations": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "Дата": {"type": "string"},
                        "Подразделение": {"type": "string"},
                        "Операция": {"type": "string"},
                        "Культура": {"type": "string"},
                        "За день (га)": {"type": "string"},
                        "С начала операции (га)": {"type": "string"},
                        "Вал за день (ц)": {"type": "string"},
                        "Вал с начала (ц)": {"type": "string"}
                    },
                    "required": [
                        "Дата", "Подразделение", "Операция", "Культура",
                        "За день (га)", "С начала операции (га)", "Вал за день (ц)", "Вал с начала (ц)"
                    ],
                    "additionalProperties": False
                }
            }
        },
        "required": ["operations"],
        "additionalProperties": False
    }

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=1500,
            temperature=0.3,
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "structured_data",
                    "schema": response_schema,
                    "strict": True 
                }
            }
        )

        structured_data = json.loads(response.choices[0].message.content)
        return json.dumps(structured_data.get("operations", []), ensure_ascii=False)

    except Exception as e:
        print(f"Ошибка при обработке сообщения ID {msg_id}: {str(e)}")
        return "[]"



In [ ]:
all_tables = []

for idx, msg in messages.iterrows():
    msg_id = msg["id"]
    msg_text = msg["Сообщения"]

    print(f"Обработка сообщения ID {msg_id}")

    response = get_structured_data(msg_text, msg_id, abbreviations)

    try:
        structured_data = json.loads(response)
    except json.JSONDecodeError as e:
        print(f"Ошибка парсинга JSON для сообщения ID {msg_id}: {response}")
        print(f"Детали ошибки: {str(e)}")
        structured_data = []

    if structured_data:
        df = pd.DataFrame(structured_data)
        df['msg id'] = msg_id
        all_tables.append(df)
    else:
        empty_row = {
            'msg id': msg_id,
            'Дата': '',
            'Подразделение': '',
            'Операция': '',
            'Культура': '',
            'За день (га)': '',
            'С начала операции (га)': '',
            'Вал за день (ц)': '',
            'Вал с начала (ц)': ''
        }
        all_tables.append(pd.DataFrame([empty_row]))

final_df = pd.concat(all_tables, ignore_index=True)

final_df = final_df[["msg id", "Дата", "Подразделение", "Операция", "Культура",
                     "За день (га)", "С начала операции (га)", "Вал за день (ц)", "Вал с начала (ц)"]]

output_file = 'structured_messages.xlsx'
final_df.to_excel(output_file, index=False, engine='openpyxl')

wb = Workbook()
ws = wb.active
ws.title = "Structured Messages"

for r in dataframe_to_rows(final_df, index=False, header=True):
    ws.append(r)

for row in range(2, len(final_df) + 2):
    if all(cell.value is None or cell.value == '' for cell in ws[row][1:]):
        for cell in ws[row]:
            cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

wb.save(output_file)
print(f"Данные успешно экспортированы в '{output_file}'")

Обработка сообщения ID 0
Обработка сообщения ID 1
Обработка сообщения ID 2
Обработка сообщения ID 3
Обработка сообщения ID 4
Обработка сообщения ID 5
Обработка сообщения ID 6
Обработка сообщения ID 7
Обработка сообщения ID 8
Обработка сообщения ID 9
Обработка сообщения ID 10
Обработка сообщения ID 11
Обработка сообщения ID 12
Обработка сообщения ID 13
Обработка сообщения ID 14
Обработка сообщения ID 15
Обработка сообщения ID 16
Обработка сообщения ID 17
Обработка сообщения ID 18
Обработка сообщения ID 19
Обработка сообщения ID 20
Обработка сообщения ID 21
Обработка сообщения ID 22
Обработка сообщения ID 23
Обработка сообщения ID 24
Обработка сообщения ID 25
Обработка сообщения ID 26
Обработка сообщения ID 27
Обработка сообщения ID 28
Обработка сообщения ID 29
Обработка сообщения ID 30
Обработка сообщения ID 31
Обработка сообщения ID 32
Обработка сообщения ID 33
Обработка сообщения ID 34
Обработка сообщения ID 35
Обработка сообщения ID 36
Обработка сообщения ID 37
Обработка сообщения ID